# Importing libraries

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Load dataset

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# scale 0–1
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# add channel dim
x_train = np.expand_dims(x_train, -1)  # (N,28,28,1)
x_test  = np.expand_dims(x_test, -1)

num_classes = 10
y_train_c = keras.utils.to_categorical(y_train, num_classes)
y_test_c  = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape, y_train_c.shape)


(60000, 28, 28, 1) (60000, 10)


# Data augmentation

In [4]:
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(x_train)


In [5]:
def build_cnn_model_v2():
    model = keras.Sequential([
        layers.Input(shape=(28, 28, 1)),

        layers.Conv2D(32, (3,3), padding="same"),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), padding="same"),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), padding="same"),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D((2,2)),

        layers.Flatten(),
        layers.Dense(256),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dropout(0.5),

        layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

cnn_model = build_cnn_model_v2()
cnn_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,330 (1.50 MB)

 Trainable params: 391,370 (1.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [7]:
batch_size = 128
epochs = 15

# Build training generator
train_gen = datagen.flow(
    x_train, 
    y_train_c, 
    batch_size=batch_size,
    shuffle=True
)

# Convert generator → tf.data.Dataset (fixes Keras 3.x warnings)
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_signature=(
        tf.TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 10), dtype=tf.float32)
    )
)

# Number of steps per epoch
steps_per_epoch = len(x_train) // batch_size

# Train model using dataset
history = cnn_model.fit(
    train_dataset,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=(x_test, y_test_c),
    callbacks=[es, rlr],
    verbose=1
)


Epoch 1/15
468/468 ━━━━━━━━━━━━━━━━━━━━ 356s 747ms/step - accuracy: 0.9767 - loss: 0.0760 - val_accuracy: 0.9818 - val_loss: 0.0619 - learning_rate: 0.0010
Epoch 2/15
468/468 ━━━━━━━━━━━━━━━━━━━━ 298s 638ms/step - accuracy: 0.9803 - loss: 0.0644 - val_accuracy: 0.9882 - val_loss: 0.0375 - learning_rate: 0.0010
Epoch 3/15
468/468 ━━━━━━━━━━━━━━━━━━━━ 1223s 3s/step - accuracy: 0.9835 - loss: 0.0546 - val_accuracy: 0.9884 - val_loss: 0.0381 - learning_rate: 0.0010
Epoch 4/15
468/468 ━━━━━━━━━━━━━━━━━━━━ 334s 714ms/step - accuracy: 0.9855 - loss: 0.0479 - val_accuracy: 0.9840 - val_loss: 0.0495 - learning_rate: 0.0010
Epoch 5/15
468/468 ━━━━━━━━━━━━━━━━━━━━ 310s 663ms/step - accuracy: 0.9885 - loss: 0.0385 - val_accuracy: 0.9907 - val_loss: 0.0291 - learning_rate: 5.0000e-04
Epoch 6/15
468/468 ━━━━━━━━━━━━━━━━━━━━ 334s 713ms/step - accuracy: 0.9886 - loss: 0.0370 - val_accuracy: 0.9916 - val_loss: 0.0251 - learning_rate: 5.0000e-04
Epoch 7/15
468/468 ━━━━━━━━━━━━━━━━━━━━ 841s 2s/step - acc

In [8]:
test_loss, test_acc = cnn_model.evaluate(x_test, y_test_c, verbose=0)
print("Test accuracy:", test_acc)

cnn_model.save("mnist_cnn_v2.keras")
print("Model saved as mnist_cnn_v2.keras")


Test accuracy: 0.9955000281333923
Model saved as mnist_cnn_v2.keras
